# 1 Libraries

In [2]:
from influxdb import DataFrameClient
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates 
import requests 
from datetime import datetime, timedelta
import time
import ast

# 2 Order Book merge functions

# 2.1 Binance

In [2]:
# EVENT_TIME (E)
def create_Binance(part, part_dif):
    # Pass numbers to float format
    part_dif.loc[:,"Seg"] = part_dif.loc[:,"Seg"].apply(lambda x: ast.literal_eval(x))
    part.loc[:, "Asks"] = part.loc[:,"Asks"].apply(lambda x: dict(list([[i[0],float(i[1])] for i in ast.literal_eval(x)])))
    part.loc[:,"Bids"] = part.loc[:,"Bids"].apply(lambda x: dict(list([[i[0],float(i[1])] for i in ast.literal_eval(x)])))
    part_dif.loc[:, "Asks"] = part_dif.loc[:,"Asks"].apply(lambda x: dict(list([[i[0],float(i[1])] for i in ast.literal_eval(x)])))
    part_dif.loc[:,"Bids"] = part_dif.loc[:,"Bids"].apply(lambda x: dict(list([[i[0],float(i[1])] for i in ast.literal_eval(x)])))

    
    # Gather first snapshot valid
    part["Type"] = "Snapshot"
    part_dif["Type"] = "Dif"
    first = 0
    for i in range(len(part)):
        for j in range(len(part_dif)):
            if part_dif.iloc[j,4][1] < int(part.iloc[i,3]) and part_dif.iloc[j,4][0] > int(part.iloc[i,3]):
                first = [i,j]
                break
            elif part_dif.iloc[j,4][1] > int(part.iloc[i,3]):
                break
        if first != 0:
            break
            
    
    # Rebuild order book for each timestemp we have
    part_join = part_dif.iloc[first[1]::,[0,1,4,5]].rename(columns = {"Seg":"LastUpdateID"}).append(part[["Asks","Bids","LastUpdateID", "Type"]].iloc[first[0],:]).sort_index()
    part_join["U_Asks"] = ""
    part_join["U_Bids"] = ""
    part_join.iat[0,4] = part_join.iloc[0,0].copy()
    part_join.iat[0,5] = part_join.iloc[0,1].copy()
    for i in range(1,len(part_join)):
        for j in range(2):
            part_join.iat[i,4+j] = part_join.iloc[i-1,4+j].copy()
            for key in part_join.iloc[i,j].keys():
                if  key in part_join.iloc[i,4+j]:
                    if part_join.iloc[i,j][key] == 0.0:
                        del part_join.iloc[i,4+j][key]
                    else:
                        part_join.iat[i,4+j][key] = part_join.iloc[i,j][key]
                else:
                    part_join.iat[i,4+j][key] = part_join.iloc[i,j][key]
                    
    return part_join

# 2.2 Bitfinex
TBD, probably will not add it due to lack of timestamp in websocket feed

In [24]:
"re".upper()

'RE'

In [21]:
datetime.fromtimestamp(float("1588940252.4267")).strftime('%Y-%m-%d %H:%M:%S')

'2020-05-08 14:17:32'

In [17]:
a = datetime.now()
str(a.date()) + "_" + str(a.second)

'2020-05-08_48'

In [ ]:
#Probable workaround https://api.bitfinex.com/v1/book/btcusd
# def create_Bitfinex(part, part_dif):
#     pass
# # First updates
# part = dfT[dfT["Host"] == "Bitfinex"]
# part_dif = d_dif[d_dif["Host"] == "Bitfinex"]
# part_dif
# part

# 2.3 Bithumb

In [6]:
# Timestamp in msg
def create_Bithumb(part, part_dif):
    # Evaluate strings
    part_dif.loc[:,"Asks"] = part_dif.loc[:,"Asks"].apply(lambda x: ast.literal_eval(x) if x != "-" else x)
    part_dif.loc[:,"Bids"] = part_dif.loc[:,"Bids"].apply(lambda x: ast.literal_eval(x) if x != "-" else x)
    part.loc[:,"Asks"] =  part.loc[:,"Asks"].apply(lambda x: ast.literal_eval(x) if x != "-" else x)
    part.loc[:,"Bids"] =  part.loc[:,"Bids"].apply(lambda x: ast.literal_eval(x) if x != "-" else x)
    
    # Create updated order book
    
    part_dif["U_Asks"] = ""
    part_dif["U_Bids"] = ""
    for i in range(len(part_dif)):
        if i == 0:
            part_dif.iat[0,5] = dict(part.iloc[0,0]).copy()
            part_dif.iat[0,6] = dict(part.iloc[0,1]).copy()
        else:
            part_dif.iat[i,5] = part_dif.iloc[i-1,5].copy()
            part_dif.iat[i,6] = part_dif.iloc[i-1,6].copy()
            for element in part_dif.iat[i,0]:
                if element != "[]":
                    if element[0] in part_dif.iloc[i,5] and element[1] == '0.000000':
                        del part_dif.iloc[i,5][element[0]]
                    else:
                        part_dif.iat[i,5][element[0]] = element[1]
            for element in part_dif.iat[i,1]:
                if element != "[]":
                    if element[0] in part_dif.iloc[i,6] and element[1] == '0.000000':
                        del part_dif.iloc[i,6][element[0]]
                    else:
                        part_dif.iat[i,6][element[0]] = element[1]
                    
    return part_dif

# 2.4 Bitstamp

In [7]:
# Timestamp in message
def create_Bitstamp(part, part_dif):
    # First updates

    part_dif = part_dif[part_dif["Pair"] == "ethbtc"]
    part_dif.loc[:,"Seg"] = part_dif.loc[:,"Seg"].apply(lambda x: int(ast.literal_eval(x)[0]))

    # Evaluate strings
    part_dif.loc[:,"Asks"] = part_dif.loc[:,"Asks"].apply(lambda x: dict(ast.literal_eval(x)) if x != "-" else x)
    part_dif.loc[:,"Bids"] = part_dif.loc[:,"Bids"].apply(lambda x: dict(ast.literal_eval(x)) if x != "-" else x)
    part.loc[:,"Asks"] =  part.loc[:,"Asks"].apply(lambda x:dict(ast.literal_eval(x)) if x != "-" else x)
    part.loc[:,"Bids"] =  part.loc[:,"Bids"].apply(lambda x: dict(ast.literal_eval(x)) if x != "-" else x)
    
    # Gather first snapshot valid
    part["Type"] = "Snapshot"
    part_dif["Type"] = "Dif"
    first = 0
    for i in range(len(part)):
        for j in range(len(part_dif)):
            if part_dif.iloc[j,4] < int(part.iloc[i,3]) and part_dif.iloc[j+1,4] > int(part.iloc[i,3]):
                first = [i,j+1]
                break
            elif part_dif.iloc[j,4] > int(part.iloc[i,3]):
                break
        if first != 0:
            break
            
    # Rebuild order book for each timestemp we have
    part_join = part_dif.iloc[first[1]::,[0,1,4,5]].rename(columns = {"Seg":"LastUpdateID"}).append(part[["Asks","Bids","LastUpdateID", "Type"]].iloc[first[0],:]).sort_index()
    part_join["U_Asks"] = ""
    part_join["U_Bids"] = ""
    part_join.iat[0,4] = part_join.iloc[0,0].copy()
    part_join.iat[0,5] = part_join.iloc[0,1].copy()
    for i in range(1,len(part_join)):
        for j in range(2):
            part_join.iat[i,4+j] = part_join.iloc[i-1,4+j].copy()
            for key in part_join.iloc[i,j].keys():
                if  key in part_join.iloc[i,4+j]:
                    if part_join.iloc[i,j][key] == '0.00000000':
                        del part_join.iloc[i,4+j][key]
                    else:
                        part_join.iat[i,4+j][key] = part_join.iloc[i,j][key]
                else:
                    part_join.iat[i,4+j][key] = part_join.iloc[i,j][key]
    
    return part_join

# 2.5 Coinbase

In [8]:
# Timestamp in message
def create_Coinbase(part, part_dif):

    # Evaluate strings
    part_dif.loc[:,"Asks"] = part_dif.loc[:,"Asks"].apply(lambda x: ast.literal_eval(x) if x != "-" else x)
    part_dif.loc[:,"Bids"] = part_dif.loc[:,"Bids"].apply(lambda x: ast.literal_eval(x) if x != "-" else x)
    part.loc[:,"Asks"] =  part.loc[:,"Asks"].apply(lambda x:dict(ast.literal_eval(x)) if x != "-" else x)
    part.loc[:,"Bids"] =  part.loc[:,"Bids"].apply(lambda x: dict(ast.literal_eval(x)) if x != "-" else x)

    part_dif.loc[:,"Seg"] = part_dif.loc[:,"Seg"].apply(lambda x: datetime.strptime(ast.literal_eval(x)[0],"%Y-%m-%dT%H:%M:%S.%fZ")  + timedelta(hours = 2))
#   In case we want to change indexes
# part_dif.index.values[i]
    # Rebuild order book for each timestemp we have

    part_dif["U_Asks"] = ""
    part_dif["U_Bids"] = ""
    part_dif.iat[0,5] = part.iloc[0,0].copy()
    part_dif.iat[0,6] = part.iloc[0,1].copy()
    
    for i in range(1,len(part_dif)):
        part_dif.iat[i,5] = part_dif.iloc[i-1,5].copy()
        part_dif.iat[i,6] = part_dif.iloc[i-1,6].copy()
        for element in part_dif.iloc[i,1]:
            if  "sell" == element[0]:
                if element[1] in part_dif.iloc[i,5]:
                    if element[2] == '0.00000000':
                        del part_dif.iloc[i,5][element[1]]
                    else:
                        part_dif.iat[i,5+j][key] = element[2]
                elif element[2] != '0.00000000':
                    part_dif.iat[i,5][key] = element[2]
            else:
                if element[1] in part_dif.iloc[i,6]:
                    if element[2] == '0.00000000':
                        del part_dif.iloc[i,6][element[1]]
                    else:
                        part_dif.iat[i,5+j][key] = element[2]
                elif element[2] != '0.00000000':
                    part_dif.iat[i,6][key] = element[2]
                    
    return part_dif

# 2.6 Huobi

In [36]:
# Timestamp in message
def create_Huobi(part, part_dif):
    # First updates
    part.loc[:,"Asks"] = part.loc[:,"Asks"].apply(lambda x: dict(ast.literal_eval(x)) if x != "-" else x)
    part.loc[:,"Bids"] = part.loc[:,"Bids"].apply(lambda x: dict(ast.literal_eval(x)) if x != "-" else x)  
    part = part.rename(columns = {"Asks": "U_Asks", "Bids": "U_Bids"})
    return part

# 2.7 Kraken

In [10]:
# Timestamp in message
# Careful with "r" for republished updates!
def create_Kraken(part, part_dif):

    part.loc[:,"Asks"] = part.loc[:,"Asks"].apply(lambda x: dict([[i[0],i[1]] for i in ast.literal_eval(x)]) if x != "-" else x)
    part.loc[:,"Bids"] = part.loc[:,"Bids"].apply(lambda x: dict([[i[0],i[1]] for i in ast.literal_eval(x)]) if x != "-" else x)  
    part_dif.loc[:,"Asks"] =  part_dif.loc[:,"Asks"].apply(lambda x: dict([[i[0],[i[1],i[2]]] for i in ast.literal_eval(x)]) if x != "-" else x)
    part_dif.loc[:,"Bids"] =  part_dif.loc[:,"Bids"].apply(lambda x: dict([[i[0],[i[1],i[2]]] for i in ast.literal_eval(x)]) if x != "-" else x)    

    # Rebuild order book for each timestemp we have

    part_dif["U_Asks"] = ""
    part_dif["U_Bids"] = ""
    part_dif.iat[0,5] = part.iloc[0,0].copy()
    part_dif.iat[0,6] = part.iloc[0,1].copy()

    for i in range(1,len(part_dif)):
        for j in range(2):
            part_dif.iat[i,5+j] = part_dif.iloc[i-1,5+j].copy()
            if part_dif.iloc[i,j] != "-":
                for key in part_dif.iloc[i,j].keys():
                    if  key in part_dif.iloc[i,5+j]:
                        if float(part_dif.iloc[i,j][key][0]) == 0.0:
                            del part_dif.iloc[i,5+j][key]
                        else:
                            part_dif.iat[i,5+j][key] = part_dif.iloc[i,j][key][0]
                    else:
                        part_dif.iat[i,5+j][key] = part_dif.iloc[i,j][key][0]

    return part_dif

# 3.- General Functions

In [11]:
def compute_stats(ex, part_dif):
    # Compute orderbook price to add to the main dataset
    first = 0
    second = 0
    part_dif["OrderBookPrice " + ex] = ""
    indb = part_dif.columns.get_loc("U_Bids")
    inda = part_dif.columns.get_loc("U_Asks")
    indf = part_dif.columns.get_loc("OrderBookPrice " + ex)
    for i in range(len(part_dif)):
        values = sorted(part_dif.iloc[i,inda].keys())
        values2 = sorted(part_dif.iloc[i,indb].keys(), reverse = True)
        for j in values:
            if float(part_dif.iloc[i,inda][j]) > 0:
                first = float(j)
                break
        for j in values2:
            if float(part_dif.iloc[i,indb][j]) > 0:
                second = float(j)
                break
        part_dif.iat[i,indf] = (first + second) / 2
    return part_dif

In [12]:
def join_main(ex, df2, part_join):
    df2 = pd.merge(df2,part_join["OrderBookPrice " + ex], how='outer', left_index=True, right_index=True)
    filter_col = [col for col in df2 if (col.startswith('Last') or col.startswith('Order'))]
    df2.update(df2[filter_col].ffill().bfill())
    filter_col = [col for col in df2 if (col.startswith('Price') or col.startswith('Q'))]
    df2.update(df2[filter_col].fillna(0))
    
    return df2

In [13]:
def create_OrderBook(ex, df2, part,part_dif):
    if ex == "Bitstamp":
        part_dif = create_Bitstamp(part, part_dif)
    elif ex == "Bitfinex":
        part_dif = create_Bitfinex(part, part_dif)
    elif ex == "Binance":
        part_dif = create_Binance(part, part_dif)
    elif ex == "Huobi":
        part_dif = create_Huobi(part, part_dif)
    elif ex == "Kraken":
        part_dif = create_Kraken(part, part_dif)
    elif ex == "Coinbase":
        part_dif = create_Coinbase(part, part_dif)
    elif ex == "Bithumb":
        part_dif = create_Bithumb(part, part_dif)
    
    part_dif = compute_stats(ex, part_dif)
    df2 = join_main(ex, df2,part_dif)
    
    return df2

In [44]:
def create_core(df, pair):
    df1 = df[df["Pair"] == pair]
    exchanges = np.unique(df1["Host"])
    df2 = df1[df1["Host"] == exchanges[0]].groupby(pd.Grouper(freq = "ms")).agg({"Price":[np.mean],"Q":[np.sum]}).fillna(method = "ffill")
    df2.columns = df2.columns.droplevel(level = 0)
    df2 = df2.rename(columns = {"mean": "LastTrade " + exchanges[0], "sum": "Q " + exchanges[0]})
    for i in exchanges[1::]:
        dft = df1[df1["Host"] == i].groupby(pd.Grouper(freq = "ms")).agg({"Price":[np.mean],"Q":[np.sum]}).fillna(method = "ffill")
        df2[["LastTrade " + i, "Q "+i]] = dft[["Price", "Q"]]
    col1 = [col for col in df2 if (col.startswith('Last'))]
    df2.update(df2[col1].ffill().bfill())
    col2 = [col for col in df2 if (col.startswith('Q'))]
    df2.update(df2[col2].fillna(0))
    return df2

# 4 Final dataframe process

## 4.1 Connect

In [45]:
client = DataFrameClient('localhost', 8086, 'root', 'root')
client.switch_database("Markets")

## 4.2 Query

In [46]:
points = client.query("SELECT * FROM Price WHERE time > '2020-05-01 17:50:00' and time < '2020-05-02 17:50:00'")
points_book = client.query("SELECT * FROM Book WHERE time > '2020-05-01 17:50:00' and time < '2020-05-02 17:50:00'")
points_dif = client.query("SELECT * FROM difBook WHERE time > '2020-05-01 17:50:00' and time < '2020-05-02 17:50:00'")

df = points["Price"]
dfT = points_book["Book"]
d_dif = points_dif["difBook"]

exchanges = np.unique(points["Price"]["Host"])

## 4.3 Main core

In [47]:
df2 = create_core(df, "ethbtc")

In [48]:
df2

,LastTrade Binance,Q Binance,LastTrade Bitfinex,Q Bitfinex,LastTrade Bithumb,Q Bithumb,LastTrade Bitstamp,Q Bitstamp,LastTrade Coinbase,Q Coinbase,LastTrade Huobi,Q Huobi,LastTrade Kraken,Q Kraken
2020-05-01 17:52:24.535000+00:00,0.024066,0.141,0.024067,0.0,0.024021,0.0,0.024028,0.0,0.02405,0.0,0.024058,0.0,0.02406,0.0
2020-05-01 17:52:24.536000+00:00,0.024066,0.000,0.024067,0.0,0.024021,0.0,0.024028,0.0,0.02405,0.0,0.024058,0.0,0.02406,0.0
2020-05-01 17:52:24.537000+00:00,0.024066,0.000,0.024067,0.0,0.024021,0.0,0.024028,0.0,0.02405,0.0,0.024058,0.0,0.02406,0.0
2020-05-01 17:52:24.538000+00:00,0.024066,0.000,0.024067,0.0,0.024021,0.0,0.024028,0.0,0.02405,0.0,0.024058,0.0,0.02406,0.0
2020-05-01 17:52:24.539000+00:00,0.024066,0.000,0.024067,0.0,0.024021,0.0,0.024028,0.0,0.02405,0.0,0.024058,0.0,0.02406,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-01 18:35:22.216000+00:00,0.024111,0.000,0.024110,0.0,0.024101,0.0,0.024028,0.0,0.02412,0.0,0.024110,0.0,0.02412,0.0
2020-05-01 18:35:22.217000+00:00,0.024111,0.000,0.024110,0.0,0.024101,0.0,0.024028,0.0,0.02412,0.0,0.024110,0.0,0.02412,0.0
2020-05-01 18:35:22.218000+00:00,0.024111,0.000,0.024110,0.0,0.024101,0.0,0.024028,0.0,0.02412,0.0,0.024110,0.0,0.02412,0.0
2020-05-01 18:35:22.219000+00:00,0.024111,0.000,0.024110,0.0,0.024101,0.0,0.024028,0.0,0.02412,0.0,0.024110,0.0,0.02412,0.0


In [49]:
df2_bckp = df2

# 4.4 Creation

In [50]:
exchanges = ["Binance", "Bithumb", "Bitstamp", "Coinbase", "Huobi", "Kraken"]
pair = "ethbtc"
for i in exchanges:
    part = dfT[dfT["Host"] == i]
    part = part[part["Pair"] == pair]
    part_dif = d_dif[d_dif["Host"] == i]
    part_dif = part_dif[part_dif["Pair"] == pair]

    df2 = create_OrderBook(i, df2, part, part_dif)
df2
# Sample to centiseconds or deciseconds to reduce weight

,LastTrade Binance,Q Binance,LastTrade Bitfinex,Q Bitfinex,LastTrade Bithumb,Q Bithumb,LastTrade Bitstamp,Q Bitstamp,LastTrade Coinbase,Q Coinbase,LastTrade Huobi,Q Huobi,LastTrade Kraken,Q Kraken,OrderBookPrice Binance,OrderBookPrice Bithumb,OrderBookPrice Bitstamp,OrderBookPrice Coinbase,OrderBookPrice Huobi,OrderBookPrice Kraken
2020-05-01 17:52:22.456000+00:00,0.024066,0.000,0.024067,0.0,0.024021,0.0,0.024028,0.0,0.02405,0.0,0.024058,0.0,0.02406,0.0,0.0240675,0.0240215,0.0240664,0.024055,0.0240625,0.02407
2020-05-01 17:52:22.552000+00:00,0.024066,0.000,0.024067,0.0,0.024021,0.0,0.024028,0.0,0.02405,0.0,0.024058,0.0,0.02406,0.0,0.0240675,0.0240215,0.0240664,0.024055,0.0240625,0.02407
2020-05-01 17:52:22.785999872+00:00,0.024066,0.000,0.024067,0.0,0.024021,0.0,0.024028,0.0,0.02405,0.0,0.024058,0.0,0.02406,0.0,0.0240675,0.0240215,0.0240664,0.024055,0.0240625,0.02407
2020-05-01 17:52:22.995000064+00:00,0.024066,0.000,0.024067,0.0,0.024021,0.0,0.024028,0.0,0.02405,0.0,0.024058,0.0,0.02406,0.0,0.0240675,0.0240215,0.0240664,0.024055,0.0240625,0.02407
2020-05-01 17:52:23.080000+00:00,0.024066,0.000,0.024067,0.0,0.024021,0.0,0.024028,0.0,0.02405,0.0,0.024058,0.0,0.02406,0.0,0.0240675,0.0240215,0.0240664,0.024055,0.0240625,0.02407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-01 18:35:22.216000+00:00,0.024111,0.000,0.024110,0.0,0.024101,0.0,0.024028,0.0,0.02412,0.0,0.024110,0.0,0.02412,0.0,0.024107,0.0241045,0.0240308,0.024055,0.024106,0.02412
2020-05-01 18:35:22.217000+00:00,0.024111,0.000,0.024110,0.0,0.024101,0.0,0.024028,0.0,0.02412,0.0,0.024110,0.0,0.02412,0.0,0.024107,0.0241045,0.0240308,0.024055,0.024106,0.02412
2020-05-01 18:35:22.218000+00:00,0.024111,0.000,0.024110,0.0,0.024101,0.0,0.024028,0.0,0.02412,0.0,0.024110,0.0,0.02412,0.0,0.024107,0.0241045,0.0240308,0.024055,0.024106,0.02412
2020-05-01 18:35:22.219000+00:00,0.024111,0.000,0.024110,0.0,0.024101,0.0,0.024028,0.0,0.02412,0.0,0.024110,0.0,0.02412,0.0,0.024107,0.0241045,0.0240308,0.024055,0.024106,0.02412


In [38]:
df2 = df2_bckp